In [1]:
%%writefile app.py
import pandas as pd
import numpy as np
import streamlit as st
from io import BytesIO
import pickle
import click
import spacy
import docx2txt
import pdfplumber
from pickle import load
import requests
import re
import os
import sklearn
import PyPDF2
import nltk
import pickle as pk
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from spacy.matcher import Matcher
from sklearn.feature_extraction.text import TfidfVectorizer

# Specify NLTK data path
# nltk_data_path = os.path.join(os.environ['APPDATA'], 'nltk_data')

# Set NLTK data path
# nltk.data.path.append(nltk_data_path)


# Now import and download the necessary NLTK resources
# nltk.download('punkt', download_dir=nltk_data_path)
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
# nltk.download('wordnet')
# # nltk.download('stopwords', download_dir=nltk_data_path)
# nltk.download('omw-1.4')

en_stopwords = set(nltk.corpus.stopwords.words('english'))

# en_stopwords = set(stopwords.words('english'))


# FUNCTIONS
def getText(filename):
    # Create empty string 
    fullText = ''
    if filename.type == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
        fullText = docx2txt.process(filename)
    else:  
        with pdfplumber.open(filename) as pdf_file:
            page = pdf_file.pages[0]
            fullText = page.extract_text()
    return fullText


def display(doc_file):
    resume = []
    if doc_file.type == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
        resume.append(docx2txt.process(doc_file))
    else:
        with pdfplumber.open(doc_file) as pdf:
            pages = pdf.pages[0]
            resume.append(pages.extract_text())
    return resume

def preprocess(sentence):
    sentence = str(sentence)
    sentence = sentence.lower()
    sentence = sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url = re.sub(r'http\S+', '', cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in en_stopwords]
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(w) for w in filtered_words]
    return " ".join(lemma_words) 



file_type = pd.DataFrame([], columns=['Uploaded File',  'Predicted Profile'])


filename = []
predicted = []

#MAX_FILE_SIZE_MB = 2  # Maximum file size allowed in MB


# Load the trained model and vectorizer
model = pickle.load(open('model.pkl','rb'))
tfidf = pickle.load(open('vectorizer.pkl','rb'))




# Streamlit app

st.title('NLP Model Deployment')
    
MAX_FILE_SIZE_MB = 2  # Maximum file size allowed in MB

upload_file = st.file_uploader('Upload Resume',type=['txt','pdf','docx'],accept_multiple_files=True)



for doc_file in upload_file:
    if doc_file is not None:
        # Check file size
        if len(doc_file.getvalue()) > MAX_FILE_SIZE_MB * 1024 * 1024:
            st.error(f"File '{doc_file.name}' exceeds the maximum size limit of {MAX_FILE_SIZE_MB} MB.")
            continue
        
        filename.append(doc_file.name)
        cleaned = preprocess(display(doc_file))
        prediction = model.predict(tfidf.transform([cleaned]))[0]
        predicted.append(prediction)
        # extText = len(doc_file.pages)

        # extText = getText(doc_file)
        # len(reader.pages) 
        
if len(predicted) > 0:
    # Define a mapping dictionary to map label encoded values to original categories
     label_mapping = {
        0: 'PeopleSoft',
        1: 'React JS Developer',
        2: 'SQL Developer',
        3: 'Workday'
        # Add more mappings as needed
     }

    
    
  
    # Predict and map the labels to their original categories
predicted_categories = [label_mapping[label] for label in predicted]

    # Update the DataFrame with the original category predictions
file_type['Uploaded File'] = filename
file_type['Predicted Profile'] = predicted_categories

    # Display the updated DataFrame
st.table(file_type.style.format())

Overwriting app.py


In [ ]:
!streamlit run app.py